## Data Processing 

In this notebook we take observations from the fully cryogenic WISE observing run (provided by Nathan Myhrvold) and filter and clean them to create a high quality sample of observations which can be used for thermal modeling.

This filtering includes:
- a minimum SNR of 4 in all four bandpasses
- quality flags should contain A, B or C
- quality flags should not contain X or U
- artifact flags should be "0000" 
- a minimum of 3 observations in each bandpass

The final subsample of observations is then crossmatched with the 2016 NEOWISE PDS release. Both the observations and the NEOWISE results are then read into a sqlite database located here: 

Finally, we repeat a similar exercise but grab all observations from the full cryogenic run that have observations in W3. This sample is also read into a database and is used to make comparisons of our W3 diameter estimators (described in Section 4 of the paper) with the published NEOWISE best-fit diameters and albedos.

In [1]:
import glob
import io
import os
import zipfile
import urllib
import sqlite3 as sql
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt

import sys
sys.path.append("..")

%matplotlib inline

In [2]:
# add PDS4 reader code (http://sbndev.astro.umd.edu/wiki/Python_PDS4_Tools)
pds4_package = "../data/PDS4_tools-1.1.zip"
if os.path.isdir("../data/PDS4_tools-1.1") is not True:
    print("Could not find PDS4 tools. Attempting download...")
    url = 'http://pdssbn.astro.umd.edu/ftp/tools/readpds_python/1.1/PDS4_tools-1.1.zip'  
    urllib.request.urlretrieve(url, pds4_package)  
    print("File succesfully downloaded.")
    
    print("Decompressing...")
    zip_ref = zipfile.ZipFile(pds4_package, 'r')
    zip_ref.extractall("../data/PDS4_tools-1.1")
    zip_ref.close()
    
else:
    print("PDS4 tools has already been downloaded...")
print("Done.")
    
sys.path.append("../data/PDS4_tools-1.1/pds4_tools-1.1/")

Could not find PDS4 tools. Attempting download...
File succesfully downloaded.
Decompressing...
Done.


In [3]:
import pds4_tools

from atm import Constants as c
from atm.obs import WISE
from atm import clipObservations
from atm import crossmatchNEOWISE

In [4]:
DO_PROCESSING = True
DATABASE = "../data/sample.db"

if DO_PROCESSING:

    ### Read in full data file
    data = pd.read_csv("../data/fullcryoobservations.csv", low_memory=False)

    ### Sort values by name and mjd
    data.sort_values(by=["nameasstring", "mjd"], inplace=True)

    ### Make same cuts as Nathan to produce a gold standard
    #-          SNR >= 4.0
    #-          Artifact flag = 0
    #-          Quality flag = A, B, C
    #-          Have at least 3 points in each of the four bands
    # See: http://wise2.ipac.caltech.edu/docs/release/prelim/expsup/sec2_2a.html
    print("Data has {} observations of {} unique asteroids.".format(len(data), len(data["nameasstring"].unique())))

    # SNR cut: above 4 in all bands (similar to setting quality flag to B) (now set to above 4.0 in all bands)
    snr = 4.0
    snr_cuts = ((data["w1snr"] >= snr) & (data["w2snr"] >= snr) & (data["w3snr"] >= snr) & (data["w4snr"] >= snr)) 

    # Artifact flags: Should be 0 across all bands
    artifact_cut = (data["cc_flags"] == "0000")

    # Quality flags: contains either A, B, C, does not contain U or X
    quality_cut = ((data["ph_qual"].str.contains("A") |
                    data["ph_qual"].str.contains("B") | 
                    data["ph_qual"].str.contains("C")) 
                   & (~data["ph_qual"].str.contains("X") 
                    & ~data["ph_qual"].str.contains("U")))
    # Make initial cut
    cut_data = data[snr_cuts & artifact_cut & quality_cut]

    # Minimum observations in bands: (after all cuts have been made should be equivalent to just removing all asteroids that occur less than 3 times)
    num_obs_per_asteroid = cut_data["nameasstring"].value_counts()
    keep = num_obs_per_asteroid[num_obs_per_asteroid >= 3]
    cut_data = cut_data[cut_data["nameasstring"].isin(keep.index)]
    cut_data = cut_data.copy()
    print("Cut data has {} observations of {} unique asteroids.".format(len(cut_data), len(cut_data["nameasstring"].unique())))


    # We need fluxes and not magnitudes to do fitting
    # Details on color correction and magnitudes: http://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html
    # Details on values Nathan had are in section 2.7, https://arxiv.org/pdf/1605.06490.pdf, the same values are also available here:
    mag_columns = ["w1mpro", "w2mpro", "w3mpro", "w4mpro"]
    magErr_columns = ["w1sigmpro", "w2sigmpro", "w3sigmpro", "w4sigmpro"]
    obs = WISE()
    
    flux_lambda = obs.convertMagToFluxLambda(cut_data[mag_columns].values)
    fluxErr_lambda = obs.convertMagErrToFluxLambdaErr(cut_data[mag_columns].values, 
                                                      cut_data[magErr_columns].values)
    cut_data["flux_W1_si"] = flux_lambda[:, 0]
    cut_data["flux_W2_si"] = flux_lambda[:, 1]
    cut_data["flux_W3_si"] = flux_lambda[:, 2]
    cut_data["flux_W4_si"] = flux_lambda[:, 3]
    cut_data["fluxErr_W1_si"] = fluxErr_lambda[:, 0]
    cut_data["fluxErr_W2_si"] = fluxErr_lambda[:, 1]
    cut_data["fluxErr_W3_si"] = fluxErr_lambda[:, 2]
    cut_data["fluxErr_W4_si"] = fluxErr_lambda[:, 3]
    cut_data["obs_id"] = np.arange(1, len(cut_data) + 1) # Add obs_id column
    
    # Rename a few columns for user-friendliness
    cut_data.rename(columns={
        "H from MPC": "H_mag",
        "G from MPC": "G",
        "ras - asteroid to sun (au)" : "r_au",
        "rao - asteroid to WISE (au)" : "delta_au", 
        "alpha - phase angle (deg)": "alpha_deg",
        "ra":  "ra_deg",
        "dec": "dec_deg",
        "sigra" : "ra_sigma_arcsec",
        "sigdec" : "dec_sigma_arcsec",
        "sigradec" : "radec_cosigma_arcsec", 
        "nameasstring": "designation"},
        inplace=True)

    # Grab columns which will be convenient for modeling 
    need = ['obs_id', 'designation', 'r_au', 'delta_au', 'alpha_deg', 'H_mag', 'G', 'mjd', 'flux_W1_si',
            'fluxErr_W1_si', 'flux_W2_si', 'fluxErr_W2_si', 'flux_W3_si',
            'fluxErr_W3_si', 'flux_W4_si', 'fluxErr_W4_si']
    # Grab remaining columns, add obs_id so a SQL join can be performed if desired
    not_needed = set(cut_data.columns.values).difference(set(need))
    not_needed.add("obs_id")
    not_needed = sorted(not_needed, key=list(not_needed).index)

    # Split data into two DataFrames and arrange columns
    observations = cut_data[need]
    additional = cut_data[not_needed]
    additional = additional[[
        'obs_id', 
        'ra_deg', 
        'ra_sigma_arcsec', 
        'ra_u',
        'dec_deg', 
        'dec_sigma_arcsec', 
        'dec_u', 
        'radec_cosigma_arcsec',
        'w1mpro', 
        'w1rchi2', 
        'w1sat',
        'w1sigmpro',
        'w1snr',
        'w2mpro',
        'w2rchi2',
        'w2sat',
        'w2sigmpro',
        'w2snr',
        'w3mpro',
        'w3rchi2',
        'w3sat',
        'w3sigmpro',
        'w3snr',
        'w4mpro',
        'w4rchi2',
        'w4sat',
        'w4sigmpro',
        'w4snr',
        'cc_flags',
        'cntr_u',
        'dist_x',
        'pang_x',
        'ph_qual',
        'na', 
        'nb',
        'sso_flg']]
    
    # Calculate magnitudes and magnitude errors from flux lambda
    mag = obs.convertFluxLambdaToMag(observations[["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"]].values)
    magErr = obs.convertFluxLambdaErrToMagErr(observations[["flux_W1_si", "flux_W2_si", "flux_W3_si", "flux_W4_si"]].values, 
                                              observations[["fluxErr_W1_si", "fluxErr_W2_si", "fluxErr_W3_si", "fluxErr_W4_si"]].values)
    mag_cols = []
    magErr_cols = []

    # Add magnitude and magnitude error columns to observations
    for i, f in enumerate(["W1", "W2", "W3", "W4"]):
        mag_col = "mag_{}".format(f)
        magErr_col = "magErr_{}".format(f)
        mag_cols.append(mag_col)
        magErr_cols.append(magErr_col)
        observations[mag_col] = mag[:, i]
        observations[magErr_col] = magErr[:, i]
    
    # Notice that we calculated magnitudes from fluxes which were calculated from original magnitudes.. this is silly.
    # Lets make sure the published magnitudes (in the additional DataFrame) are equivalent to the converted magnitudes
    # in the observations DataFrame
    pd.testing.assert_frame_equal(additional[mag_columns], observations[mag_cols].rename(columns={c1 : c2 for c1, c2 in zip(mag_cols, mag_columns)}))
    
    # Clip observations: add a "keep" column -- any observations that are over a magnitude
    # away from a linear best fit model for each asteroid have keep set to 0. If any object
    # has more than 42% of its observations that do not satisfy the cut criteria set keep to 0. 
    # If any object has less than three observations at the end of fitting the linear model set
    # keep to 0 for all its observations.
    # 42% was chosen to make sure that object 90367 was accepted. It fails at 40%.
    # Note that if a single filters observation at an epoch is flagged, all other observations
    # at the same epoch is also flagged, this may mean this clipping is more agressive than desired.
    observations_flagged = clipObservations(observations, obs, magCut=1.0, minObs=3, maxObsCut=0.42)

    # Add NEOWISE 2016 data to database
    structure_lists = glob.glob("../data/neowise_diameters_albedos_V1_0/data/neowise_*.xml")
    neowisev1 = []

    for sl in structure_lists:
        neowise_data = pds4_tools.pds4_read(sl)
        neowisev1.append(pd.DataFrame(neowise_data["TABLE"].data))

    neowisev1 = pd.concat(neowisev1, sort=False)
    neowisev1.sort_values(by="ASTEROID_NUMBER", inplace=True)
    neowisev1.reset_index(drop=True, inplace=True)
    
    # Attempts to crossmatch observations of minor planets with the NEOWISE results table. Adds a 'matched_designation' column to
    # the NEOWISE results table with the matched designation from the observations table if found, if observations could not be matched
    #to an asteroid in the NEOWISE table then 'matched_designation' will be set to NaN.
    crossmatchNEOWISE(neowisev1, observations_flagged)
    
    # Save both DataFrames as tables in a sqlite database
    con = sql.connect(DATABASE)
    observations_flagged.to_sql("observations", con, index=False)
    additional.to_sql("additional", con, index=False)
    neowisev1.to_sql("neowise_v1", con, index=False)
    
# Read data from database
con = sql.connect(DATABASE)
observations = pd.read_sql("""SELECT * FROM observations""", con)
additional = pd.read_sql("""SELECT * FROM additional""", con)

Data has 2542956 observations of 150928 unique asteroids.
Cut data has 87293 observations of 9672 unique asteroids.


/Users/moeyensj/software/anaconda3/envs/atm_py36/lib/python3.6/site-packages/ipykernel_launcher.py:142: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/moeyensj/software/anaconda3/envs/atm_py36/lib/python3.6/site-packages/ipykernel_launcher.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Number of observations that were cut: 12344 (14.14%)
Number of observations that survived cut: 74949 (85.86%)
Number of unique objects that were cut: 2309 (23.87%)
Number of unique objects that survived cut: 7363 (76.13%)
The observations DataFrame has been updated: see observations['keep'] for clipping flag.
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_centaurs.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_irreg_sat.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_hildas.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_neos.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_mainbelt.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neo

In [5]:
DO_W3_PROCESSING = True
DATABASE_W3= "../data/sample_W3.db"

if DO_W3_PROCESSING is True:
    ### Read in full data file
    data = pd.read_csv("../data/fullcryoobservations.csv", low_memory=False)

    ### Sort values by name and mjd
    data.sort_values(by=["nameasstring", "mjd"], inplace=True)

    ### Make same cuts as Nathan to produce a gold standard
    #-          SNR >= 4.0
    #-          Artifact flag = 0
    #-          Quality flag = A, B, C
    #-          Have at least 3 points in each of the four bands
    # See: http://wise2.ipac.caltech.edu/docs/release/prelim/expsup/sec2_2a.html
    print("Data has {} observations of {} unique asteroids.".format(len(data), len(data["nameasstring"].unique())))

    # SNR cut: above 4 in all bands (similar to setting quality flag to B) (now set to above 4.0 in all bands)
    snr_W3 = 4.0
    snr_cuts_W3 = (data["w3snr"] >= snr_W3)

    # Artifact flags: Should be 0 across all bands
    artifact_flags_W3 = data["cc_flags"].str.slice(2, 3)
    artifact_cut_W3 = (artifact_flags_W3  == "0")

    # W3 only quality cut
    quality_flag_W3 = data["ph_qual"].str.slice(2, 3)
    quality_cut_W3 = ((quality_flag_W3.str.contains("A") |
                    quality_flag_W3.str.contains("B") | 
                    quality_flag_W3.str.contains("C")) 
                   & (~quality_flag_W3.str.contains("X") 
                    & ~quality_flag_W3.str.contains("U")))

    # Make initial cut
    cut_data_W3 = data[snr_cuts_W3 & artifact_cut_W3 & quality_cut_W3]

    # Minimum observations in bands: (after all cuts have been made should be equivalent to just removing all asteroids that occur less than 3 times)
    num_obs_per_asteroid = cut_data_W3["nameasstring"].value_counts()
    keep = num_obs_per_asteroid[num_obs_per_asteroid >= 3]
    cut_data_W3 = cut_data_W3[cut_data_W3["nameasstring"].isin(keep.index)]
    cut_data_W3 = cut_data_W3.copy()
    print("Cut data has {} observations of {} unique asteroids.".format(len(cut_data_W3), len(cut_data_W3["nameasstring"].unique())))

    # We need fluxes and not magnitudes to do fitting
    # Details on color correction and magnitudes: http://wise2.ipac.caltech.edu/docs/release/allsky/expsup/sec4_4h.html
    # Details on values Nathan had are in section 2.7, https://arxiv.org/pdf/1605.06490.pdf, the same values are also available here:
    mag_columns = ["w1mpro", "w2mpro", "w3mpro", "w4mpro"]
    magErr_columns = ["w1sigmpro", "w2sigmpro", "w3sigmpro", "w4sigmpro"]
    obs = WISE()
    
    flux_lambda = obs.convertMagToFluxLambda(cut_data_W3[mag_columns].values)
    fluxErr_lambda = obs.convertMagErrToFluxLambdaErr(cut_data_W3[mag_columns].values, cut_data_W3[magErr_columns].values)
    cut_data_W3["flux_W3_si"] = flux_lambda[:, 2]
    cut_data_W3["fluxErr_W3_si"] = fluxErr_lambda[:, 2]
    cut_data_W3["mag_W3"] = cut_data_W3["w3mpro"]
    cut_data_W3["magErr_W3"] = cut_data_W3["w3sigmpro"]
    cut_data_W3["obs_id"] = np.arange(1, len(cut_data_W3) + 1) # Add obs_id column
    
    # Rename a few columns for user-friendliness
    cut_data_W3.rename(columns={
        "H from MPC": "H_mag",
        "G from MPC": "G",
        "ras - asteroid to sun (au)" : "r_au",
        "rao - asteroid to WISE (au)" : "delta_au", 
        "alpha - phase angle (deg)": "alpha_deg",
        "ra":  "ra_deg",
        "dec": "dec_deg",
        "sigra" : "ra_sigma_arcsec",
        "sigdec" : "dec_sigma_arcsec",
        "sigradec" : "radec_cosigma_arcsec", 
        "nameasstring": "designation"},
        inplace=True)

    # Grab columns which will be convenient for modeling 
    need = ['obs_id', 'designation', 'r_au', 'delta_au', 'alpha_deg', 'H_mag', 'G', 'mjd', 'flux_W3_si',
            'fluxErr_W3_si', "mag_W3", "magErr_W3"]
    # Grab remaining columns, add obs_id so a SQL join can be performed if desired
    not_needed = set(cut_data_W3.columns.values).difference(set(need))
    not_needed.add("obs_id")
    not_needed = sorted(not_needed, key=list(not_needed).index)

    # Split data into two DataFrames and arrange columns
    observations = cut_data_W3[need]
    additional = cut_data_W3[not_needed]
    additional = additional[[
        'obs_id', 
        'ra_deg', 
        'ra_sigma_arcsec', 
        'ra_u',
        'dec_deg', 
        'dec_sigma_arcsec', 
        'dec_u', 
        'radec_cosigma_arcsec', 
        'w1mpro', 
        'w1rchi2', 
        'w1sat',
        'w1sigmpro',
        'w1snr',
        'w2mpro',
        'w2rchi2',
        'w2sat',
        'w2sigmpro',
        'w2snr',
        'w3mpro',
        'w3rchi2',
        'w3sat',
        'w3sigmpro',
        'w3snr',
        'w4mpro',
        'w4rchi2',
        'w4sat',
        'w4sigmpro',
        'w4snr',
        'cc_flags',
        'cntr_u',
        'dist_x',
        'pang_x',
        'ph_qual',
        'na', 
        'nb',
        'sso_flg']]
    
    # Add NEOWISE 2016 data to database
    structure_lists = glob.glob("../data/neowise_diameters_albedos_V1_0/data/neowise_*.xml")
    neowisev1 = []

    for sl in structure_lists:
        neowise_data = pds4_tools.pds4_read(sl)
        neowisev1.append(pd.DataFrame(neowise_data["TABLE"].data))

    neowisev1 = pd.concat(neowisev1, sort=False)
    neowisev1.sort_values(by="ASTEROID_NUMBER", inplace=True)
    neowisev1.reset_index(drop=True, inplace=True)
    
    # Attempts to crossmatch observations of minor planets with the NEOWISE results table. Adds a 'matched_designation' column to
    # the NEOWISE results table with the matched designation from the observations table if found, if observations could not be matched
    #to an asteroid in the NEOWISE table then 'matched_designation' will be set to NaN.
    crossmatchNEOWISE(neowisev1, observations)
    
    # Save both DataFrames as tables in a sqlite database
    con = sql.connect(DATABASE_W3)
    observations.to_sql("observations", con, index=False)
    additional.to_sql("additional", con, index=False)
    neowisev1.to_sql("neowise_v1", con, index=False)
    
# Read data from database
con = sql.connect(DATABASE_W3)
observations = pd.read_sql("""SELECT * FROM observations""", con)
additional = pd.read_sql("""SELECT * FROM additional""", con)

Data has 2542956 observations of 150928 unique asteroids.
Cut data has 2273455 observations of 149191 unique asteroids.
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_centaurs.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_irreg_sat.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_hildas.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_neos.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_mainbelt.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_ambos.xml
Now processing a Table_Character structure: TABLE
Processing label: ../data/neowise_diameters_albedos_V1_0/data/neowise_jupiter_trojans.xml
